In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
import tensorflow as tf
from tensorflow import keras

import pandas as pd
from pandas import read_csv

from sklearn.decomposition import PCA
import umap

import datetime

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers, initializers, regularizers, models, callbacks
import tensorflow.keras.backend as K

# fix CUDNN_STATUS_INTERNAL_ERROR
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

from sklearn.model_selection import train_test_split

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
# Convert from 64 to 32 bit floats
X_test = np.load("/home/alon/Projects/project/X_test.npy").astype(np.float32)
y_test = np.load("/home/alon/Projects/project/y_test.npy").astype(np.float32)
person_train_valid = np.load("/home/alon/Projects/project/person_train_valid.npy").astype(np.float32)
X_train_valid = np.load("/home/alon/Projects/project/X_train_valid.npy").astype(np.float32)
y_train_valid = np.load("/home/alon/Projects/project/y_train_valid.npy").astype(np.float32)
person_test = np.load("/home/alon/Projects/project/person_test.npy").astype(np.float32)

# adjust labels 
y_train_valid -= 769
y_test -= 769

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))
print('y_train_valid', y_train_valid[:10])

In [ ]:
# #import numpy
# # x is your dataset
# #x = numpy.random.rand(100, 5)
# indices = numpy.random.permutation(x.shape[0])
# training_idx, test_idx = indices[:80], indices[80:]
# training, test = x[training_idx,:], x[test_idx,:]

In [ ]:
#split training and validation data:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.2, random_state=42)
#X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.2, random_state=42)

print ('Training/Valid data shape: {}'.format(X_train.shape))
print ('Validation data shape: {}'.format(X_valid.shape))
print ('Training/Valid target shape: {}'.format(y_train.shape))
print ('Validation target shape: {}'.format(y_valid.shape))

In [ ]:
def ksquare(x):
    return tf.pow(x, 2)

def klog(x):
    return tf.math.log(x)

In [ ]:
def convert_to_one_hot(in_arr):
    in_arr = in_arr.reshape((in_arr.shape[0],))
    in_arr = in_arr.astype(int)
    in_arr_1h = np.zeros((in_arr.size, in_arr.max()+1))
    in_arr_1h[np.arange(in_arr.size),in_arr] = 1
    return in_arr_1h

In [ ]:
#convert all vectors to one-hot
y_train_valid_1h = convert_to_one_hot(y_train_valid)
y_train_1h = convert_to_one_hot(y_train)
y_valid_1h = convert_to_one_hot(y_valid)
y_test_1h = convert_to_one_hot(y_test)

person_train_valid_1h = convert_to_one_hot(person_train_valid)
person_test_1h = convert_to_one_hot(person_test)

In [ ]:
#normalizing and then adding noise to every single training input

X_train_valid_norm = np.zeros_like(X_train_valid)

for eeg_ix in range(X_train_valid.shape[0]):
    for i in range(22):
        #normalize the data
        y = X_train_valid[eeg_ix,i,:]
        mean = np.mean(y)
        std = np.std(y)
        z = (y-mean)/std
        X_train_valid_norm[eeg_ix,i,:] = z
        

#don't want to actually use the one with noise        
# mu, sigma = 0, 0.10
# s = np.random.normal(mu, sigma, X_train_valid_norm.shape)
# print("Noise shape: {}".format(s.shape))
# X_train_valid_norm_noised = X_train_valid_norm + s

In [ ]:
#normalizing and then adding noise to every single training input

X_train_norm = np.zeros_like(X_train)

for eeg_ix in range(X_train.shape[0]):
    for i in range(22):
        #normalize the data
        y = X_train[eeg_ix,i,:]
        mean = np.mean(y)
        std = np.std(y)
        z = (y-mean)/std
        X_train_norm[eeg_ix,i,:] = z

In [ ]:
#validation data as well
X_valid_norm = np.zeros_like(X_valid)

for eeg_ix in range(X_valid.shape[0]):
    for i in range(22):
        #normalize the data
        y = X_valid[eeg_ix,i,:]
        mean = np.mean(y)
        std = np.std(y)
        z = (y-mean)/std
        X_valid_norm[eeg_ix,i,:] = z

In [ ]:
#normalizing and then adding noise to every single test input

X_test_norm = np.zeros_like(X_test)

for eeg_ix in range(X_test.shape[0]):
    for i in range(22):
        #normalize the data
        y = X_test[eeg_ix,i,:]
        mean = np.mean(y)
        std = np.std(y)
        z = (y-mean)/std
        X_test_norm[eeg_ix,i,:] = z
        
# mu, sigma = 0, 0.10
# s = np.random.normal(mu, sigma, X_test_norm.shape)
# print("Noise shape: {}".format(s.shape))
# X_test_norm_noised = X_test_norm + s

In [ ]:
#Crop for 12

X_train_valid_norm_cropped = X_train_valid_norm[:,:,:500]
X_train_norm_cropped = X_train_norm[:,:,:500]
X_valid_norm_cropped = X_valid_norm[:,:,:500]
X_test_norm_cropped = X_test_norm[:,:,:500]

## This is the preprocessing setup for the current best network done: standardized inputs and cropped.

## Now let's try dimensionality reduction, first using PCA, and then UMAP

In [ ]:
#Take a look at some random capture

fig, axs = plt.subplots(11,2,figsize=(20,40))

count = 0

eeg_ix = 10

for i in range(22):
    y = X_train_valid_norm_cropped[eeg_ix,i,:]
    
    axs[int(i/2),count].plot(y)
    axs[int(i/2),count].set_title('EEG Channel {}, standardized and cropped'.format(i+1))
    
    count += 1
    if (count>1):
        count=0

In [ ]:
n_comp = 18

#flip
X_train_flipped = np.transpose(X_train_norm_cropped, (0,2,1))
X_valid_flipped = np.transpose(X_valid_norm_cropped, (0,2,1))
X_test_flipped = np.transpose(X_test_norm_cropped, (0,2,1))

print(X_train_flipped.shape)
print(X_valid_flipped.shape)
print(X_test_flipped.shape)

X_train_reshaped = X_train_flipped.reshape((X_train_norm_cropped.shape[0]*X_train_norm_cropped.shape[2], 22)) 
X_valid_reshaped = X_valid_flipped.reshape((X_valid_norm_cropped.shape[0]*X_valid_norm_cropped.shape[2], 22)) 
X_test_reshaped = X_test_flipped.reshape((X_test_norm_cropped.shape[0]*X_test_norm_cropped.shape[2], 22)) 

In [ ]:
#try a PCA on the training data
pca = PCA(n_components=n_comp)                  #create the pca object       
pca.fit(X_train_reshaped)                               #fit it to your transformed data
X_train_transformed=np.empty([X_train_norm_cropped.shape[0],X_train_norm_cropped.shape[2],n_comp])
for i in range(len(X_train_flipped)):
    #print(X_train_valid_flipped[i].shape)
    X_train_transformed[i]=pca.transform(X_train_flipped[i])           #iteratively apply the transformation to each instance of the original dataset

print(X_train_transformed.shape)
X_train_transformed = np.transpose(X_train_transformed, (0,2,1))
print(X_train_transformed.shape)

In [ ]:
#now apply that transformation to the validation and test sets
X_valid_transformed=np.empty([X_valid_norm_cropped.shape[0],X_valid_norm_cropped.shape[2],n_comp])
X_test_transformed=np.empty([X_test_norm_cropped.shape[0],X_test_norm_cropped.shape[2],n_comp])

for i in range(len(X_valid_flipped)):
    #print(X_train_valid_flipped[i].shape)
    X_valid_transformed[i]=pca.transform(X_valid_flipped[i])           #iteratively apply the transformation to each instance of the original dataset

#print(X_train_transformed.shape)
X_valid_transformed = np.transpose(X_valid_transformed, (0,2,1))
print(X_valid_transformed.shape)

for i in range(len(X_test_flipped)):
    #print(X_train_valid_flipped[i].shape)
    X_test_transformed[i]=pca.transform(X_test_flipped[i])           #iteratively apply the transformation to each instance of the original dataset

#print(X_train_transformed.shape)
X_test_transformed = np.transpose(X_test_transformed, (0,2,1))
print(X_test_transformed.shape)

In [ ]:
for drp in np.arange(0.7, 0.9, 0.01):
    print(drp)

In [ ]:
test_accuracies_final_1 = []
test_accuracies_best_val = []

for drp in np.arange(0.7, 0.9, 0.01):
    for n_comp in range(6, 22):
        print("Testing PCA Network for dropout = {} and n_dims = {}".format(drp,n_comp))

        #try a PCA on the data
        #reshaped_data = X_train_valid_flipped.reshape((X_train_valid_cropped.shape[1]*X_train_valid_cropped.shape[2], 22))    # create one big data panel with 20 series and 300.000 datapoints
                                              #choose the number of features to have after dimensionality reduction
        pca = PCA(n_components=n_comp)                  #create the pca object       
        pca.fit(X_train_reshaped)                               #fit it to your transformed data
        X_train_transformed=np.empty([X_train_norm_cropped.shape[0],X_train_norm_cropped.shape[2],n_comp])
        for i in range(len(X_train_flipped)):
            #print(X_train_valid_flipped[i].shape)
            X_train_transformed[i]=pca.transform(X_train_flipped[i])           #iteratively apply the transformation to each instance of the original dataset


        #print(X_train_valid_transformed.shape)    #we end up with examples*timesteps*channels
        #now transpose back to examples*channels*timesteps
        X_train_transformed = np.transpose(X_train_transformed, (0,2,1))


        #now apply that transformation to the validation and test sets
        X_valid_transformed=np.empty([X_valid_norm_cropped.shape[0],X_valid_norm_cropped.shape[2],n_comp])
        X_test_transformed=np.empty([X_test_norm_cropped.shape[0],X_test_norm_cropped.shape[2],n_comp])

        for i in range(len(X_valid_flipped)):
            #print(X_train_valid_flipped[i].shape)
            X_valid_transformed[i]=pca.transform(X_valid_flipped[i])           #iteratively apply the transformation to each instance of the original dataset

        #print(X_train_transformed.shape)
        X_valid_transformed = np.transpose(X_valid_transformed, (0,2,1))
        #print(X_valid_transformed.shape)

        for i in range(len(X_test_flipped)):
            #print(X_train_valid_flipped[i].shape)
            X_test_transformed[i]=pca.transform(X_test_flipped[i])           #iteratively apply the transformation to each instance of the original dataset

        #print(X_train_transformed.shape)
        X_test_transformed = np.transpose(X_test_transformed, (0,2,1))

        input_ = layers.Input(shape=(n_comp, 500))
        r1 = layers.Reshape(target_shape=(n_comp, 500, 1))(input_)
        c1 = layers.Conv2D(filters=40, kernel_size=(1,25), data_format='channels_last',
                           activation='elu', kernel_regularizer='l2')(r1)
        p1 = layers.Permute(dims=(2,1,3))(c1)
        r2 = layers.Reshape((476, n_comp*40))(p1)
        d1 = layers.Dense(40, activation='elu')(r2)
        sq1 = layers.Activation(ksquare)(d1)
        ap1 = layers.AveragePooling1D(75, strides=15)(sq1)
        log1 = layers.Activation(klog)(ap1)
        f1 = layers.Flatten()(log1)
        d2 = layers.Dropout(drp)(f1)
        output_ = layers.Dense(4, activation='softmax', kernel_regularizer='l2', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01))(d2)

        model = models.Model(inputs=input_, outputs=output_, name='shallow_pca_convnet_one_hot')
        #opt = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
        #model.summary()

        mcp_save = callbacks.ModelCheckpoint('.mdl_wts_best_acc' + str(n_comp) + '_dims.hdf5', save_best_only=True, monitor='val_acc', mode='max')

        loss_hist = model.fit(X_train_transformed, y_train_1h, epochs=150,
                          validation_data=(X_valid_transformed, y_valid_1h),
                          callbacks=[mcp_save], 
                          verbose=True)

        hist = loss_hist.history

        fig, axs = plt.subplots(1,2, figsize=(20, 12))

        fig.suptitle("Training results for dropout={} and n_dims={}".format(drp, n_comp), fontsize=14)

        axs[0].plot(hist['loss'])
        axs[0].plot(hist['val_loss'])
        axs[0].set_ylabel('loss')
        axs[0].set_xlabel('epoch')
        axs[0].legend(['train', 'val'])

        axs[1].plot(hist['acc'])
        axs[1].plot(hist['val_acc'])
        axs[1].set_ylabel('accuracy')
        axs[1].set_xlabel('epoch')
        axs[1].legend(['train', 'val'])

        # Evaluate the model on the test data using `evaluate`
        print("Evaluate final step on test data")
        results = model.evaluate(X_test_transformed, y_test_1h, batch_size=128)
        print("final test loss, test acc:", results)

        test_accuracies_final_1.append(results[1])

        model.load_weights('.mdl_wts_best_acc' + str(n_comp) + '_dims.hdf5')

        # Evaluate the model on the test data using `evaluate`
        print("Evaluate best val acc model on test data")
        results = model.evaluate(X_test_transformed, y_test_1h, batch_size=128)
        print("best val acc model test loss, test acc:", results)

        test_accuracies_best_val.append(results[1])


In [ ]:
test_accuracies_final_2 = []
test_accuracies_best_loss = []

for drp in np.arange(0.7, 0.9, 0.01):
    for n_comp in range(6, 22):
        print("Testing PCA Network for dropout = {} and n_dims = {}".format(drp,n_comp))

        #try a PCA on the data
        #reshaped_data = X_train_valid_flipped.reshape((X_train_valid_cropped.shape[1]*X_train_valid_cropped.shape[2], 22))    # create one big data panel with 20 series and 300.000 datapoints
                                              #choose the number of features to have after dimensionality reduction
        pca = PCA(n_components=n_comp)                  #create the pca object       
        pca.fit(X_train_reshaped)                               #fit it to your transformed data
        X_train_transformed=np.empty([X_train_norm_cropped.shape[0],X_train_norm_cropped.shape[2],n_comp])
        for i in range(len(X_train_flipped)):
            #print(X_train_valid_flipped[i].shape)
            X_train_transformed[i]=pca.transform(X_train_flipped[i])           #iteratively apply the transformation to each instance of the original dataset


        #print(X_train_valid_transformed.shape)    #we end up with examples*timesteps*channels
        #now transpose back to examples*channels*timesteps
        X_train_transformed = np.transpose(X_train_transformed, (0,2,1))


        #now apply that transformation to the validation and test sets
        X_valid_transformed=np.empty([X_valid_norm_cropped.shape[0],X_valid_norm_cropped.shape[2],n_comp])
        X_test_transformed=np.empty([X_test_norm_cropped.shape[0],X_test_norm_cropped.shape[2],n_comp])

        for i in range(len(X_valid_flipped)):
            #print(X_train_valid_flipped[i].shape)
            X_valid_transformed[i]=pca.transform(X_valid_flipped[i])           #iteratively apply the transformation to each instance of the original dataset

        #print(X_train_transformed.shape)
        X_valid_transformed = np.transpose(X_valid_transformed, (0,2,1))
        #print(X_valid_transformed.shape)

        for i in range(len(X_test_flipped)):
            #print(X_train_valid_flipped[i].shape)
            X_test_transformed[i]=pca.transform(X_test_flipped[i])           #iteratively apply the transformation to each instance of the original dataset

        #print(X_train_transformed.shape)
        X_test_transformed = np.transpose(X_test_transformed, (0,2,1))

        input_ = layers.Input(shape=(n_comp, 500))
        r1 = layers.Reshape(target_shape=(n_comp, 500, 1))(input_)
        c1 = layers.Conv2D(filters=40, kernel_size=(1,25), data_format='channels_last',
                           activation='elu', kernel_regularizer='l2')(r1)
        p1 = layers.Permute(dims=(2,1,3))(c1)
        r2 = layers.Reshape((476, n_comp*40))(p1)
        d1 = layers.Dense(40, activation='elu')(r2)
        sq1 = layers.Activation(ksquare)(d1)
        ap1 = layers.AveragePooling1D(75, strides=15)(sq1)
        log1 = layers.Activation(klog)(ap1)
        f1 = layers.Flatten()(log1)
        d2 = layers.Dropout(0.80)(f1)
        output_ = layers.Dense(4, activation='softmax', kernel_regularizer='l2', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01))(d2)

        model = models.Model(inputs=input_, outputs=output_, name='shallow_pca_convnet_one_hot')
        #opt = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
        #model.summary()

        mcp_save = callbacks.ModelCheckpoint('.mdl_wts_best_loss' + str(n_comp) + '_dims.hdf5', save_best_only=True, monitor='val_loss', mode='min')

        loss_hist = model.fit(X_train_transformed, y_train_1h, epochs=150,
                          validation_data=(X_valid_transformed, y_valid_1h),
                          callbacks=[mcp_save], 
                          verbose=True)

        hist = loss_hist.history

        fig, axs = plt.subplots(1,2, figsize=(20, 12))

        fig.suptitle("Training results for dropout={} and n_dims={}".format(drp, n_comp), fontsize=14)

        axs[0].plot(hist['loss'])
        axs[0].plot(hist['val_loss'])
        axs[0].set_ylabel('loss')
        axs[0].set_xlabel('epoch')
        axs[0].legend(['train', 'val'])

        axs[1].plot(hist['acc'])
        axs[1].plot(hist['val_acc'])
        axs[1].set_ylabel('accuracy')
        axs[1].set_xlabel('epoch')
        axs[1].legend(['train', 'val'])

        # Evaluate the model on the test data using `evaluate`
        print("Evaluate final step on test data")
        results = model.evaluate(X_test_transformed, y_test_1h, batch_size=128)
        print("final test loss, test acc:", results)

        test_accuracies_final_2.append(results[1])

        model.load_weights('.mdl_wts_best_loss' + str(n_comp) + '_dims.hdf5')

        # Evaluate the model on the test data using `evaluate`
        print("Evaluate best val loss model on test data")
        results = model.evaluate(X_test_transformed, y_test_1h, batch_size=128)
        print("best val loss model test loss, test acc:", results)

        test_accuracies_best_loss.append(results[1])

In [ ]:
input_ = layers.Input(shape=(n_comp, 500))
r1 = layers.Reshape(target_shape=(n_comp, 500, 1))(input_)
c1 = layers.Conv2D(filters=40, kernel_size=(1,25), data_format='channels_last',
                   activation='elu', kernel_regularizer='l2')(r1)
p1 = layers.Permute(dims=(2,1,3))(c1)
r2 = layers.Reshape((476, n_comp*40))(p1)
d1 = layers.Dense(40, activation='elu')(r2)
sq1 = layers.Activation(ksquare)(d1)
ap1 = layers.AveragePooling1D(75, strides=15)(sq1)
log1 = layers.Activation(klog)(ap1)
f1 = layers.Flatten()(log1)
d2 = layers.Dropout(0.80)(f1)
output_ = layers.Dense(4, activation='softmax', kernel_regularizer='l2', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01))(d2)

In [ ]:
model = models.Model(inputs=input_, outputs=output_, name='shallow_convnet_one_hot')
#opt = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
## Training 12

## Wow, best performance now 74%

earlyStopping = callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=True, restore_best_weights=True, mode='min')
mcp_save = callbacks.ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_acc', mode='min')
#thresh_cb = MyThresholdCallback(threshold=0.73)
#reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

#loss_hist = model.fit(Xtr_more, Ytr_more,epochs=50, verbose=0, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.25)


loss_hist = model.fit(X_train_transformed, y_train_1h, epochs=100,
                      validation_data=(X_valid_transformed, y_valid_1h),
                      callbacks=[mcp_save], 
                      verbose=True)

hist = loss_hist.history

plt.subplot(1,2,1)
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'])

plt.subplot(1,2,2)
plt.plot(hist['acc'])
plt.plot(hist['val_acc'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'])

In [ ]:
hist = loss_hist.history

fig, axs = plt.subplots(1,2, figsize=(20, 12))

fig.suptitle("Training results", fontsize=14)

axs[0].plot(hist['loss'])
axs[0].plot(hist['val_loss'])
axs[0].set_ylabel('loss')
axs[0].set_xlabel('epoch')
axs[0].legend(['train', 'val'])

axs[1].plot(hist['acc'])
axs[1].plot(hist['val_acc'])
axs[1].set_ylabel('accuracy')
axs[1].set_xlabel('epoch')
axs[1].legend(['train', 'val'])

In [ ]:
#model.load_weights('.mdl_wts.hdf5')

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test_transformed, y_test_1h, batch_size=128)
print("test loss, test acc:", results)


In [ ]:
model.load_weights('.mdl_wts.hdf5')

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test_transformed, y_test_1h, batch_size=128)
print("test loss, test acc:", results)

In [ ]:
print(results[1])